In [1]:
import pandas as pd
data = pd.read_excel("../../../db/clean_orj_ulakbim.xlsx")
data.head()

,text,label
0,türk çimento sektörü ekonomik sosyal çevresel ...,Muhendislik
1,kollu galeri açma makineleri madencilik tünelc...,Muhendislik
2,özet araştırma derlenen bilgiler i̇stanbul ili...,Muhendislik
3,özet araştırma derlenen bilgiler i̇stanbul ili...,Muhendislik
4,özet çalışmada karadeniz yakalanan istavrit tr...,Muhendislik


In [2]:
data = data.dropna()

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2999 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2999 non-null   object
 1   label   2999 non-null   object
dtypes: object(2)
memory usage: 70.3+ KB


In [4]:
import gensim
review_text = data.text.apply(gensim.utils.simple_preprocess)
review_text

0       [türk, çimento, sektörü, ekonomik, sosyal, çev...
1       [kollu, galeri, açma, makineleri, madencilik, ...
2       [özet, araştırma, derlenen, bilgiler, stanbul,...
3       [özet, araştırma, derlenen, bilgiler, stanbul,...
4       [özet, çalışmada, karadeniz, yakalanan, istavr...
                              ...                        
2995    [çalışma, heracleum, platytaenium, boiss, öğre...
2996    [korunan, alanlar, ilgili, çalışmalar, eski, z...
2997    [çalışmada, bölgeye, uyumlarını, belirlemek, a...
2998    [çalışma, balıkesir, yöresindeki, sanayi, doma...
2999    [vertisol, latince, vertere, dönen, anlamına, ...
Name: text, Length: 2999, dtype: object

In [5]:
from gensim.models.doc2vec import TaggedDocument
class TaggedDocumentIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc, tags=[self.labels_list[idx]])
 
docLabels = data.label.tolist()
data_text = review_text.tolist()
sentences = TaggedDocumentIterator(data_text, docLabels)

In [6]:
model = gensim.models.Doc2Vec(
    vector_size=100,
    window=10,
    min_count=2,
    alpha=0.025,
    workers=11,
    epochs=20
)

In [7]:
model.build_vocab(sentences)

In [8]:
model.epochs

20

In [9]:
model.train(sentences,total_examples=model.corpus_count,epochs=model.epochs)

In [10]:
model.save("d2v_orj_ulakbim.doc2vec")